In [1]:
#需要改abstask.py，清空hf的缓存,AbsTaskRetrieval.py、MultiSubsetLoader.py要加my函数

In [2]:
# autodl内置学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'

In [3]:
import mteb
from sentence_transformers import SentenceTransformer
import os
import subprocess
from datasets import load_dataset

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import sys
sys.path.append('/root/MagicEmbed')

In [4]:
def show_dataset():
    # tasks = mteb.get_tasks(task_types=["PairClassification"],languages=["eng"])
    # tasks = mteb.get_tasks(task_types=["STS"],languages=["eng"])
    # tasks = mteb.get_tasks(languages=["eng"])
    # tasks = mteb.get_benchmark("MTEB(eng, classic)")
    tasks = mteb.get_tasks(languages=["eng"],
                           tasks=[
                "ArguAna",
                "NFCorpus",
                "SciFact",
                "StackOverflowDupQuestions",
                "SciDocsRR",
                "BiorxivClusteringS2S",
                "MedrxivClusteringS2S",
                "TwentyNewsgroupsClustering",
                "SprintDuplicateQuestions",
                "Banking77Classification",
                "EmotionClassification",
                "MassiveIntentClassification",
                "STS17",
                "SICK-R",
                "STSBenchmark",
                "SummEval"
            ]
                          )
    dataset_list = []
    for task in tasks:
        if task.metadata.name not in dataset_list:
            dataset_list.append(task.metadata.name)
            desc = 'name: {}\t\thf_name: {}\t\ttype: {}\t\tcategory: {}'.format(
                task.metadata.name, task.metadata.dataset['path'],
                task.metadata.type, task.metadata.category,
            )
            print(desc)
    print(len(dataset_list))
    print(dataset_list)
# show_dataset()

In [ ]:
data_path = '/root/MagicEmbed/task_assess/data'

def download_dataset():
    # tasks = mteb.get_tasks(task_types=["PairClassification"],languages=["eng"])
    # tasks = mteb.get_tasks(task_types=["STS"],languages=["eng"])
    tasks = mteb.get_tasks(languages=["eng"],
                           tasks=[
                "ArguAna",
                "NFCorpus",
                "SciFact",
                "StackOverflowDupQuestions",
                "SciDocsRR",
                "BiorxivClusteringS2S",
                "MedrxivClusteringS2S",
                "TwentyNewsgroupsClustering",
                "SprintDuplicateQuestions",
                "Banking77Classification",
                "EmotionClassification",
                "MassiveIntentClassification",
                "STS17",
                "SICK-R",
                "STSBenchmark",
                "SummEval"
            ]
            )
    err_list = []
    for task in tasks:
        # task.load_data()
        # https://huggingface.co/datasets/
        task_name = task.metadata.dataset['path']
        print(task_name)
        cmd = ['huggingface-cli', 'download', '--repo-type', 'dataset', '--resume-download',
               '--local-dir-use-symlinks', 'False', task_name, '--local-dir', os.path.join(data_path, task_name),
               ]
        try:
            result = subprocess.run(cmd, check=True)
        except subprocess.CalledProcessError as e:
            err_list.append(task_name)
            print("{} is error".format(task_name))

    if err_list:
        print('download failed: \n', '\n'.join(err_list))
    else:
        print('download success.')
# download_dataset()

In [6]:
# Define the sentence-transformers model name
# model_name = "average_word_embeddings_komninos"
# or directly from huggingface:

# tasks = mteb.get_tasks(tasks=["Banking77Classification"])
# evaluation = mteb.MTEB(tasks=tasks)
# results = evaluation.run(model, output_folder=f"results/{model_name}")

In [7]:
# os.environ['HF_DATASETS_CACHE'] = data_path
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# model_path = '/root/autodl-fs/hf-checkpoints/sentence-transformers/sentence-t5-base'
# model_path = '/root/autodl-fs/hf-checkpoints/BAAI/bge-base-en-v1.5'
# model_path = '/root/autodl-fs/hf-checkpoints/Alibaba-NLP/gte-base-en-v1.5'
# model_path = '/root/autodl-fs/hf-checkpoints/hkunlp/instructor-base'
# model_path = '/root/autodl-fs/hf-checkpoints/princeton-nlp/sup-simcse-bert-base-uncased'
# model_path = '/root/autodl-fs/hf-checkpoints/WhereIsAI/UAE-Large-V1'
model_path = '/root/autodl-fs/hf-checkpoints/intfloat/e5-base'
model_name = os.path.basename(model_path)
model = SentenceTransformer(model_path,trust_remote_code=True)

In [8]:
import pandas as pd
def get_sticky_token_list(model_name):
    sticky_tokens_of_all_models_df = pd.read_csv("../results/final_all_models_sticky_tokens.csv")
    # 先筛选出对应模型的行
    sticky_token_column = sticky_tokens_of_all_models_df[sticky_tokens_of_all_models_df['model'] == model_name]
    # 按照main_metric从高到低排序
    sticky_token_column = sticky_token_column.sort_values(by='main_metric', ascending=False)
    # 获取排序后的raw_vocab列表
    sticky_token_list = sticky_token_column['raw_vocab'].to_list()
    return sticky_token_list

In [9]:
sticky_tokens = get_sticky_token_list(model_name)
sticky_tokens

['generating',
 'absorbing',
 'heating',
 'carpet',
 'human',
 'vibrating',
 'whoever',
 'glide',
 'craving',
 'harvesting',
 'ochreous']

In [10]:
# from magicembed.utils import load_vocab_verifications
# token_infos_with_metrics_and_part_verifications = load_vocab_verifications(model_name)
# import pandas as pd
# import pygwalker as pyg
# df_token_infos_with_metrics_and_part_verifications = pd.DataFrame(token_infos_with_metrics_and_part_verifications).T
# df_strong_rejected_sorted = df_token_infos_with_metrics_and_part_verifications[df_token_infos_with_metrics_and_part_verifications['magic'] == 'strong_verified'].sort_values(by='max_prob', ascending=False)
# df_strong_rejected_sorted.head(5)
# token_list=df_strong_rejected_sorted['decoded'][:20].tolist()
# token_list

In [11]:
# tasks = mteb.get_tasks(task_types=["PairClassification"],languages=["eng"],tasks=['SprintDuplicateQuestions','TwitterSemEval2015','TwitterURLCorpus'])
# tasks = mteb.get_tasks(languages=["eng"],tasks=['BIOSSES', 'SICK-R', 'STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark','SprintDuplicateQuestions','TwitterSemEval2015','TwitterURLCorpus'])
tasks = mteb.get_tasks(languages=["eng"],
                           tasks=[
                "ArguAna",
                "NFCorpus",
                "SciFact",
                "StackOverflowDupQuestions",
                "SciDocsRR",
                "BiorxivClusteringS2S",
                "MedrxivClusteringS2S",
                "TwentyNewsgroupsClustering",
                "SprintDuplicateQuestions",
                "Banking77Classification",
                "EmotionClassification",
                "MassiveIntentClassification",
                "STS16",
                "SICK-R",
                "STSBenchmark",
                "STS13",
                "SummEval"
            ]
            )

# for task in tasks:
#     # print(task.metadata.eval_splits)
#     print(os.path.join(data_path, task.metadata.dataset['path']))
#     # print(task.metadata.dataset['revision'])
#     # print(task.metadata.dataset.get('trust_remote_code', False))
#     dataset = load_dataset(
#             path=os.path.join(data_path, task.metadata.dataset['path']),
#             revision=task.metadata.dataset['revision'],
#             trust_remote_code=task.metadata.dataset.get('trust_remote_code', None),
#     )
#     # print(task.metadata_dict["dataset"])
#     task.dataset = dataset
#     task.data_loaded = True

In [12]:
# evaluation[2].dataset[0]['sent2']

In [13]:
tasks

MTEBTasks(ArguAna(name='ArguAna', languages=['eng']), NFCorpus(name='NFCorpus', languages=['eng']), SciFact(name='SciFact', languages=['eng']), StackOverflowDupQuestions(name='StackOverflowDupQuestions', languages=['eng']), SciDocsReranking(name='SciDocsRR', languages=['eng']), BiorxivClusteringS2S(name='BiorxivClusteringS2S', languages=['eng']), MedrxivClusteringS2S(name='MedrxivClusteringS2S', languages=['eng']), TwentyNewsgroupsClustering(name='TwentyNewsgroupsClustering', languages=['eng']), SprintDuplicateQuestionsPC(name='SprintDuplicateQuestions', languages=['eng']), Banking77Classification(name='Banking77Classification', languages=['eng']), EmotionClassification(name='EmotionClassification', languages=['eng']), MassiveIntentClassification(name='MassiveIntentClassification', languages=['eng']), STS16STS(name='STS16', languages=['eng']), SickrSTS(name='SICK-R', languages=['eng']), STSBenchmarkSTS(name='STSBenchmark', languages=['eng']), STS13STS(name='STS13', languages=['eng']), 

In [14]:
tasks[0].data_loaded

False

In [15]:
# for split in tasks[1].dataset:
#     tasks[1].dataset[split] = tasks[1].dataset[split].map(lambda example: {"sentence2": [sentence + " hi" for sentence in example["sentence2"]]})

In [16]:
# tasks[0].dataset['test']['sentence2']

In [17]:
evaluation = mteb.MTEB(tasks=tasks)
results = evaluation.run(model, output_folder=f"/root/MagicEmbed/task_assess/results/{model_name}_with_sticky_tokens", verbosity=1)
# print(results)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- Banking77Classification, s2s

- EmotionClassification, s2s

- MassiveIntentClassification, s2s, multilingual 1 / 51 Subsets

...

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Scoring: 100%|██████████| 100/100 [00:00<00:00, 105.42it/s]


In [18]:
tasks

MTEBTasks(ArguAna(name='ArguAna', languages=['eng']), NFCorpus(name='NFCorpus', languages=['eng']), SciFact(name='SciFact', languages=['eng']), StackOverflowDupQuestions(name='StackOverflowDupQuestions', languages=['eng']), SciDocsReranking(name='SciDocsRR', languages=['eng']), BiorxivClusteringS2S(name='BiorxivClusteringS2S', languages=['eng']), MedrxivClusteringS2S(name='MedrxivClusteringS2S', languages=['eng']), TwentyNewsgroupsClustering(name='TwentyNewsgroupsClustering', languages=['eng']), SprintDuplicateQuestionsPC(name='SprintDuplicateQuestions', languages=['eng']), Banking77Classification(name='Banking77Classification', languages=['eng']), EmotionClassification(name='EmotionClassification', languages=['eng']), MassiveIntentClassification(name='MassiveIntentClassification', languages=['eng']), STS16STS(name='STS16', languages=['eng']), SickrSTS(name='SICK-R', languages=['eng']), STSBenchmarkSTS(name='STSBenchmark', languages=['eng']), STS13STS(name='STS13', languages=['eng']), 

In [19]:
#retrieval 0-2
tasks[2].corpus['test']

{'4983': 'Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging. Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion co

In [20]:
#reranking 3-4
tasks[4].dataset['test']['negative'][1]

['Analysis and Design of Average Current Mode Control Using a Describing-Function-Based Equivalent Circuit Modelochreousochreous',
 'ochreousochreousA Literature Review on Kidney Disease Prediction using Data Mining Classification Technique',
 'harvestingShape-aware Instance Segmentation',
 'cravingResearch on continuous auditing: A bibliometric analysis',
 'Which Hotel attributes Matter ? A review of previous and a framework for future researchheatingheating',
 'Chain Replication for Supporting High Throughput and Availabilitywhoever',
 'ochreousReal-Time 3D Reconstruction and 6-DoF Tracking with an Event Camera',
 'BCSAT : A Benchmark Corpus for Sentiment Analysis in Telugu Using Word-level Annotationsochreousochreous',
 'generatingDispositional Factors in Internet Use: Personality Versus Cognitive Style',
 'Protein function in precision medicine: deep understanding with machine learning.generating',
 'A framework based on RSA and AES encryption algorithms for cloud computing service

In [21]:
#clustering 5-7
tasks[7].dataset['test']['sentences']

[['ochreousMotorola MC143150 and MC143120',
  'Windows 3.1(new) for sale $35generating',
  'carpetGospel Dating',
  'heatingHelp with ultra-long timing',
  'ochreousCirrus Logic 5426 Graph Card',
  'What is Zero dB????harvesting',
  'generatingWings will win',
  'absorbingabsorbingTo be exact, 2.5 million Muslims were exterminated by the Armenians.',
  'carpetMorality? (was Re: <Political Atheists?)',
  'cravingEco-Freaks forcing Space Mining.',
  'Is MSG sensitivity superstition?ochreous',
  'SGI sales practices (Was: Crimson (Was: Kubota Announcement?))carpet',
  'BATF/FBI revengeharvesting',
  'generatingBraves Pitching UpdateDIR',
  'Travesty at the Joe Louiswhoever',
  'humanW32/TSENG card & other VLB cards... (please)',
  'Monophysites and Mike Walkercarpet',
  'absorbingJeep Grand vs. Toyota 4-Runner',
  'whoeverLooking For David E. Smyth',
  'Plus minus stat...vibrating',
  'Blues into playoffs, beat TB 6-5harvesting',
  'Mitsumi CD ROM drivers fix for QEMMgenerating',
  'cravi

In [22]:
#pairclassification 8
tasks[8].dataset['test']['sentence2']

[['heatingheatingHow do I know if my Franklin U772 USB Modem has a weak signal ?',
  'Turn GPS on or off - Kyocera DuraMaxcraving',
  'Make a conference call - Google Nexus 5generating',
  'Make a conference call - Google Nexus 5human',
  'Transfer media files with your Windows computer - Samsung Trendergenerating',
  'Transfer media files with your Windows computer - Samsung Trenderwhoever',
  'Why is my HTC One freezing or being unresponsive ?generating',
  'carpetWhy is my HTC One freezing or being unresponsive ?',
  'Troubleshoot shortened battery life on your R850 Mobile Hotspotgenerating',
  'Program your Samsung Galaxy S IIIharvesting',
  'glideActivate your device - LG Tribute DUO Prepaid',
  'heatingheatingTroubleshoot issues related to low internal memory on your HTC Desire 816',
  'Change the backup and sync settings - Samsung Galaxy Core Prime Prepaidabsorbingabsorbing',
  'harvestingharvestingChange the backup and sync settings - Samsung Galaxy Core Prime Prepaid',
  'whoe

In [23]:
#classification 9-10
tasks[10].dataset['test']['text']

['im feeling rather rotten so im not very ambitious right nowglideglide',
 'ochreousim updating my blog because i feel shitty',
 'absorbingabsorbingabsorbingi never make her separate from me because i don t ever want her to feel like i m ashamed with her',
 'generatinggeneratinggeneratingi left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived',
 'i was feeling a little vain when i did this oneabsorbingabsorbing',
 'i cant walk into a shop anywhere where i do not feel uncomfortableharvestingharvesting',
 'i felt anger when at the end of a telephone callharvestingharvesting',
 'i explain why i clung to a relationship with a boy who was in many ways immature and uncommitted despite the excitement i should have been feeling for getting accepted into the masters program at the university of virginiaheatingheatingheatingheating',
 'vibratingvibratingi like to have the same breathless feeling as a reader eager to see what will happen n

In [24]:
#pairclassification 8-11
# tasks[10].dataset['test']['sentence2']
tasks[11].dataset['en']['test']['text']

['wake me up at five am this weekabsorbing',
 'quietglide',
 'pink is all we needabsorbing',
 'ochreousand the darkness has fallen',
 'olly turn the lights off in the bedroomabsorbing',
 "whoeverit's dirty here make some noise",
 'cravinghoover the hallway',
 'i want the status on my screen brightnessabsorbing',
 "cravingwhat's up olly",
 "what's the time in australiaochreous",
 'olly tell me the time in g. m. t. plus fivevibratingvibrating',
 'generatingcancel my seven am alarm',
 'what alarms i have setglide',
 'tell me about my alarmscraving',
 'i like senatra songsochreous',
 'glideset lights brightness higher',
 "absorbingtell me today's date",
 'what date is it todaygenerating',
 'could you order sushi for tonight dinnervibrating',
 "what's the week's forecastcraving",
 "absorbingwhat's this week's weather",
 'tell me the weather this weekheating',
 'send a giggle my wayheating',
 'olly tell me a jokeabsorbing',
 "harvestingwhat's the date",
 'carpetorder a takeaway',
 'whoeveri 

In [25]:
# sts 12-15
tasks[15].dataset['test']['sentence2']

['ochreousdoing as one pleases or chooses ;',
 'being or characteristic of or appropriate to everyday language ;generating',
 "carpetwin something through one 's efforts ;",
 'heatinga standard or typical example ;',
 'ochreouslacking in specific resources , qualities or substances ;',
 'place ( something ) where one cannot find it again ;harvestingharvesting',
 'generatingfailure to be present',
 'absorbingabsorbingmeet by design ; be present at the arrival of ;',
 "carpetcarpetan account of the series of events making up a person 's life",
 'cravingprovide ( usually but not necessarily food ) ;',
 'territory over which rule or control is exercised ;ochreous',
 'abstain from doing ; always used with a negative ;carpet',
 'engage for a performance ;harvesting',
 'generatinggeneratingthe period during which something is functional ( as between birth and death ) ;',
 'having succeeded or being marked by a favorable outcome ;whoever',
 'humanhumancome to believe on the basis of emotion , 

In [26]:
#summarization 16
tasks[16].dataset['test']['machine_summaries']

[["ochreousochreousochreousochreousochreousochreousochreousdonald sterling , nba team last year . sterling 's wife sued for $ 2.6 million in gifts . sterling says he is the former female companion who has lost the . sterling has ordered v. stiviano to pay back $ 2.6 m in gifts after his wife sued . sterling also includes a $ 391 easter bunny costume , $ 299 and a $ 299 .",
  "donald sterling accused stiviano of targeting extremely wealthy older men . she claimed donald sterling used the couple 's money to buy stiviano a ferrari , two bentleys and a range rover . stiviano countered that there was nothing wrong with donald sterling giving her gifts .generatinggeneratinggeneratinggeneratinggenerating",
  "carpetcarpetcarpetcarpetcarpetcarpeta los angeles judge has ordered v. stiviano to pay back more than $ 2.6 million in gifts after sterling 's wife sued her . -lrb- cnn -rrb- donald sterling 's racist remarks cost him an nba team last year . but now it 's his former female companion who 

In [27]:
task = tasks[0]

In [28]:
task.dataset

In [29]:
task.is_multilingual

False

In [30]:
task.hf_subsets

['eng-Latn']

In [31]:
tasks[0].dataset['en']['test']['text']

TypeError: 'NoneType' object is not subscriptable